<a href="https://colab.research.google.com/github/daniilprigozhin/Soy_NLRome/blob/main/Soy_CladeFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Dependencies
##This block takes 4-5 minutes
import pandas as pd
import numpy as np
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c bioconda hmmer 
!conda install -c bioconda mafft
!pip install Bio
from collections import defaultdict
import pandas as pd
from Bio import SearchIO
!conda install -c bioconda epa-ng
!conda install -c bioconda gappa
from google.colab import files

In [ ]:
#@title Select Species
Species = "Soy" #@param ["Soy", "Arabidopsis", "Maize"]

if Species == 'Soy':
  !rm -rf Soy_NLRome/

  !git clone https://github.com/daniilprigozhin/Soy_NLRome.git

  %cd Soy_NLRome

  common = pd.read_csv("Soy_NLRome_GeneTable.txt", sep='\t')


clades = common[["Clade", "File"]].groupby("Clade").first()
!mkdir Clade_HMM
for row in clades.iterrows():
  !hmmbuild -n {row[0]} Clade_HMM/{row[0]}.hmm {row[1]["File"]}

!cat Clade_HMM/*.hmm > All_Clades.hmm

In [ ]:
#@title Code from exporting/importing to gdrive. Unnecessary Right now.
# #export
# from google.colab import drive
# drive.mount('/content/drive')
# !mkdir /content/drive/MyDrive/Soy_NLRome_Colab
# !cp -r ../Soy_NLRome/ /content/drive/MyDrive/Soy_NLRome_Colab

# #import
# from google.colab import drive

# drive.mount('/content/drive')
# !cd ./drive/MyDrive/Soy_NLRome_Colab/Soy_NLRome

In [ ]:
# Code used from https://github.com/sokrypton/ColabFold
import re
import hashlib

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]

query_sequence = 'MAWGSSSSSSSSSSNYDVFLSFRGEDTRHAFTGHLYKALHDKGIHTFIDDEKLQRGEQITRALMEAIQDSRVAITVLSQNYASSSFCLDELATILHCHQRKRLLVIPVFYKVDPSDVRHQKGSYAEALEKLETRFQHDPEKLQKWKMALKQVADLSGYHFKEGDGYEFKFIEKIVERVSREINPRTLHVADYPVGLESRVLDVRRLLDAGSDDGVHMIGIHGMGGLGKSTLARAVYNELIIAEKFDGFCFLANVREKSDKKDGLEHLQRILLSEILGEKNISLTSTQQGISIIQSRLKGKKVLLILDDVNTHGQLQAIGRRDWFGPGSKIIITTRDEQLLAYHEVNETYEMKELNQKDALQLLTWNAFKKEKADPTYVEVLHRVVAYASGLPLALEVIGSHLVGKSIEAWESAIKQYKRIPKKEILDVLTVSFDALEEEEQKVFLDIACCLKGWTLTEVEHILPGLYDDCMKHNIGVLVEKSLIKVSWGDGVVNMHDLIQDMGRRIDQQRSSKEPGKRRRLWLTKDIIQVLDDNSKFRKLKVLKFDYCKILTEIPDVSVLVNLEELSFNRCGNLITVHHSIGFLNKLKILSAYGCSKLTTFPPLNLTSLEGLQLSACSSLENFPEILGEMKNLLMLQLFGLLGVKELPVSFQNLVGLQSLILQDCENFLLPSNIIAMMPKLSSLLAESCKGLQWVKSEEGEEKVGSIVCSNVDDSSFDGCNLYDDFFSTGFMQLDHVKTLSLRDNNFTFLPECLKELQFLTRLDVSGCLRLQEIRGVPPNLKEFMARECISLSSSSSSMLSNQELHEAGQTEFLFPGATIPEWFNHQSRGPSSSFWFRNKFPDNVLCLLLARVESIDLDDIPMPKVFINGILCKISSRNYQVRKVKLDYTYLFDLKSALYKLDDPSGLISALHELDEKEWDHVEITYGGIIETSLLKATGIHVFRQDDIRYDDPYGKRKLEHDLNSSESQSLIKKPRLSRWVGPERIINLLGNAADGALFTNPLRRYRHTSMDSSCPRCPELEETCLHALRDCPKVAAFWRSVLPKKLAPKFFNGDVAVWLETNLSFSEAAFFWPTFFGIAVELLWESRNDLVFYKDGTWDYLDLSDITDIVFNRYKDCMRAHASHILMPRNLLKWRRPLPLSHGHWLLRLNVSGAYDRSSDTAACGGIFRDNNDRFVLGFSVKLGECLSNDEGEIWGIYHGMKIARRYDIWGLYRDILRQHSIPISAKIARQHDFGKIIVESGSEKAIGFVLDGCPTSTSLEHCFPLCDELKALTSATNHLYFDNDYAADSFAKFGLSMKRQPVKIFRVCPPFCQPYISEYR' #@param {type:"string"}
# remove whitespaces
query_sequence = "".join(query_sequence.split())
query_sequence = re.sub(r'[^a-zA-Z]','', query_sequence).upper()

jobname = 'test' #@param {type:"string"}
# remove whitespaces
jobname = "".join(jobname.split())
jobname = re.sub(r'\W+', '', jobname)
jobname_unhashed = jobname
jobname = add_hash(jobname, query_sequence)


with open(f"{jobname}.fasta", "w") as text_file:
    text_file.write(">" + jobname_unhashed + "\n%s" % query_sequence)

In [ ]:
#@title Create an Annotated Tree with your protein included
!hmmsearch --domtblout {jobname}.tbl.out All_Clades.hmm {jobname}.fasta 

filename = jobname + ".tbl.out"

attribs = ['bias', 'bitscore',  'evalue']

hits = defaultdict(list)
names = []

with open(filename) as handle:
    for queryresult in SearchIO.parse(handle, 'hmmsearch3-domtab'):
      #print(queryresult.id)
      #print(queryresult.accession)
      #print(queryresult.description)
      for hit in queryresult.hsps:
        names.append(queryresult.id)
        for attrib in attribs:
          hits[attrib].append(getattr(hit, attrib))

final = pd.DataFrame.from_dict(hits)
final["Clade"] = names
final.sort_values("bitscore", ascending = False)

#common = pd.read_csv("Soy_NLRome_GeneTable.txt", sep='\t')

final_hv = final.set_index("Clade").join(common[["Clade", "HV", "File"]].groupby("Clade").agg(max), how = 'left', rsuffix = "_other").sort_values("bitscore", ascending = False)
best_afa = final_hv.iloc[0,:]["File"]
!mafft --add {jobname}.fasta --keeplength {best_afa} > {jobname}.updated.afa

clade_oi = common[common["File"] == best_afa].iloc[0,:]["Clade"]
tree_oi = best_afa[:best_afa.find("Int")] + "RAxML_bestTree." + clade_oi + ".Raxml.out"
raxml_info_oi = best_afa[:best_afa.find("Int")] + "RAxML_info." + clade_oi + ".Raxml.out"

!epa-ng --split {best_afa} {jobname}.updated.afa --tree {tree_oi} --model {raxml_info_oi}
!cut -f1 -d ' ' {best_afa} > best_afa_f.afa
!epa-ng --ref-msa best_afa_f.afa --tree {tree_oi} -q query.fasta --model {raxml_info_oi} --redo

!gappa examine graft --jplace-path ./

tree_created = True

is_hv = final_hv.iloc[0,:]["HV"] == 1

if is_hv:
  print("Your protein belongs to a highly variable clade.")

Above ends the process of tree creation. Below we create a Shannon Entropy model (receommended for proteins in highly variable clades).

In [ ]:
#@title Create a Shannon Entropy Reading compared to other Proteins in Same Clade
!Rscript Minimal_Chimera_Entropy_Script.R -f {jobname}.updated.afa -n test
shannon_created = True
#ls ./Test/test_/ PATH TO ENTROPY READINGS

In [ ]:
#@title Download Results
#@markdown Select which options you prefer and run. If save to drive option is chosen, the result zip will be uploaded to your Google Drive.
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: epa_result.jplace/newick for the tree, and {jobname}/{jobname} for the Shannon Entropy, right-click and select "Download".


download_tree = False #@param {type:"boolean"}

download_shannon_entropy = False #@param {type:"boolean"}

download_to_drive = False #@param {type:"boolean"}

if (download_tree and tree_created):
  if (download_shannon_entropy and shannon_created):
    #BUGGY PART HERE, IDK WHAT SHANNON ENTROPY FILES ARE CALLED
    !zip -FSr $jobname".result.zip" "epa_result.newick" "epa_result.jplace" $jobname/jobname*/jobname*"ChimeraEntropy.txt" $jobname/jobname*/jobname*"Entropy_MaskedNG.png"
    files.download(f"{jobname}.result.zip")
    if download_to_drive == True and drive:
      uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
      uploaded.SetContentFile(f"{jobname}.result.zip")
      uploaded.Upload()
      print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")
  else:
    !zip -FSr $jobname".result.zip" "epa_result.newick" "epa_result.jplace" 
    files.download(f"{jobname}.result.zip")
    if download_to_drive == True and drive:
      uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
      uploaded.SetContentFile(f"{jobname}.result.zip")
      uploaded.Upload()
      print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")
  
